# IMEC2201 Herramientas Computacionales 
## Taller Semana 6: Optimización
### Fecha Límite Entrega (Bloque Neón): <font color="#FF5733">23.59h de Mayo 22, 2022</font>

Universidad de los Andes — Mayo 11, 2022.
___
### Tener en Cuenta
La entrega del Taller 6 se puede realizar de tres maneras:
1. Cargar la solución en su repositorio en GitHub y enviar el enlace del mismo vía Bloque Neón.
2. Enviar vía Bloque Neón el archivo en Jupyter Notebook (extensión '.ipynb').
3. Enviar vía Bloque Neón el archivo PDF del Jupyter Notebook.
___

# Semana 6: Optimizacion

En esta semana se trabajaran diferentes casos de optimización, desde bombas hasta turbinas eólicas. Este tipo de ejercicios se realizan en ingeniería mecánica para el diseño de diferentes sistemas de bombeo o de conversión de energía. En complemento a lo visto en clase, se dejara un taller práctico con el fin de optimizar diferentes parámetros de bombas y turbinas eólicas. 

In [15]:
using Pkg

# Importar paquetes
Pkg.add("JuMP")
Pkg.add("GLPK")
Pkg.add("Ipopt")
Pkg.add("Plots")
Pkg.add("Dierckx")
Pkg.add("DataFrames")
Pkg.add("Interpolations")
Pkg.add("Roots") 
Pkg.add("NLsolve")

Pkg.status()



      Status `~/Documents/GitHub/Matias_Cadena/Project.toml`
  [a93c6f00] DataFrames v1.3.4
  [39dd38d3] Dierckx v0.5.2
  [60bf3e95] GLPK v1.0.1
  [a98d9a8b] Interpolations v0.13.6
  [b6b21f68] Ipopt v1.0.2
  [4076af6c] JuMP v1.0.0
  [2774e3e8] NLsolve v4.5.1
  [91a5bcdd] Plots v1.29.0
  [f2b01f46] Roots v2.0.1


   Resolving package versions...
  No Changes to `~/Documents/GitHub/Matias_Cadena/Project.toml`
  No Changes to `~/Documents/GitHub/Matias_Cadena/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Documents/GitHub/Matias_Cadena/Project.toml`
  No Changes to `~/Documents/GitHub/Matias_Cadena/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Documents/GitHub/Matias_Cadena/Project.toml`
  No Changes to `~/Documents/GitHub/Matias_Cadena/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Documents/GitHub/Matias_Cadena/Project.toml`
  No Changes to `~/Documents/GitHub/Matias_Cadena/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Documents/GitHub/Matias_Cadena/Project.toml`
  No Changes to `~/Documents/GitHub/Matias_Cadena/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Documents/GitHub/Matias_Cadena/Project.toml`
  No Changes to `~/Documents/GitHub/Matias_Cadena/Manifest.toml`
   Resolving package version

In [16]:
using JuMP
using GLPK
using Ipopt
using Plots
using Dierckx
using DataFrames
using Interpolations
using Roots 
using NLsolve 

# Bomba Centrifuga

Se necesita conocer la cabeza máxima de una bomba ficticia, cuya eficiencia es constante en un 30%. Opera entre 1000 y 2000 RPM, y el diámetro mínimo es de 10 centímetros. 

**Nota:** Buscar las propiedades hidráulicas del agua a temperatura ambiente.

#### Detalles
Recuerden que la eficiencia de la bomba es:

$$
\eta = \frac{P_w}{P_f} = \frac{\rho g Q H}{w T}
$$

Donde la potencia que la bomba centrífuga le añade al fluido (conocida como **potencia hidráulica** $P_w$), es:

$$
P_w = \rho g Q H
$$

Y para que la bomba centrífuga le añada energía al fluido, debe haber un recurso que, asimismo, le suministre energía al dispositivo. Esto último es conocido como **potencia mecánica** $P_f$ y es el producto entre la velocidad de rotación del eje de la bomba $w$ y el torque en el mismo $T$.

$$
P_f = w T
$$

También:

$$
Q = VA
$$

y

$$
A = \frac{\pi D^2}{4}
$$

In [47]:
function bomba_centrifuga()
    n = .30 # eficiencia %
    p = 997 # Densidad del agua kg/m³
    g = 9.81 # gravedad en m/s²
    model = Model(Ipopt.Optimizer)
    
    @variable(model, 1000 <= w <= 2000)
    @variable(model, 0.1 <= D)
    @variable(model, T >= 0)
    @variable(model, v >= 0)
    @variable(model, H >= 0)
    
    @NLobjective(model, Max, (w*T*n)/(p*g*v*pi*D^2/4))

    @NLconstraint(model, c1, n == (p*g*v*pi*D^2/4*H) / (w * T))

    print(model)
    optimize!(model)
    
    @show objective_value(model)
    @show value(w)
    @show value(T)
    @show value(D)
    @show value(v)
    return # Poner respuesta final aqui
end

bomba_centrifuga()

Max (w * T * 0.3) / ((997.0 * 9.81 * v * 3.141592653589793 * D ^ 2.0) / 4.0)
Subject to
 w ≥ 1000.0
 D ≥ 0.1
 T ≥ 0.0
 v ≥ 0.0
 H ≥ 0.0
 w ≤ 2000.0
 0.3 - (((997.0 * 9.81 * v * 3.141592653589793 * D ^ 2.0) / 4.0) * H) / (w * T) = 0
This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        5
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       25

Total number of variables............................:        5
                     variables with only lower bounds:        4
                variables with lower and upper bounds:        1
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:  

objective_value(model) = 30958.29075222165
value(w) = 1184.5471209909986
value(T) = 64.42199719049492
value(D) = 0.20023354545727864
value(v) = 0.002401060396146639


# Turbina Eólica

Para un parque eólico en La Guajira, se tomaron medidas de velocidad del viento, las cuales, segun el sitio https://www.windy.com/ oscilan entre 15 y 22 nudos. Si se planea que la velocidad angular de la turbina no supere las 3000 RPM, encontrar el mínimo diámetro posible con el fin de extraer 60 GW de potencia del viento. 

**Nota:** Tener en cuenta que la potencia eólica es: $P_w = \frac{1}{2}\rho A v^3$ donde $\rho$ es la densidad del aire. 

#### Detalles
Recuerde que:
- 1 nudo equivale a 0.51 m/s.
- La densidad $\rho$ del aire es 1.225 kg/m$^3$.

También, el área $A$ es:

$$
A = \frac{\pi D^2}{4}
$$

In [51]:
function turbina_eolica()
    p = 1.225 #kg/m³
    nudos = 15:22 #Nudos
    vel = nudos*0.51 #velocidad
    
    model = Model(Ipopt.Optimizer)
    
    @variable(model, vel[1] <= v <= last(vel))
    @variable(model, D >= 0)
    @variable(model, A >= 0)
    
    @NLobjective(model, Min, sqrt(4*A/pi))
    
    @NLconstraint(model, c1, 0.5*p*pi/4*(D^2)*(v^3) >= 6*10^9)
    @NLconstraint(model, c2, A == pi*D^2/4)
    
    print(model)
    optimize!(model)
    
    @show objective_value(model)
    @show value(v)

    return # De nuevo, escribir la respuesta final aquí
end

turbina_eolica()

Min sqrt((4.0 * A) / 3.141592653589793)
Subject to
 v ≥ 7.65
 D ≥ 0.0
 A ≥ 0.0
 v ≤ 11.22
 ((0.5 * 1.225 * 3.141592653589793) / 4.0) * D ^ 2.0 * v ^ 3.0 - 6.0 * 10.0 ^ 9.0 ≥ 0
 A - (3.141592653589793 * D ^ 2.0) / 4.0 = 0
This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        2
Number of nonzeros in inequality constraint Jacobian.:        2
Number of nonzeros in Lagrangian Hessian.............:        5

Total number of variables............................:        3
                     variables with only lower bounds:        2
                variables with lower and upper bounds:        1
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        1
   inequality constraints with lower and upper bounds:        0
   